# Análisis Completo de Regresión Lineal

Este notebook muestra paso a paso cómo realizar un análisis de **Regresión Lineal** empleando un conjunto de datos que contiene:
- **valuationdate**: fecha de valoración (opcional para el modelo)
- **revenues**: ingresos
- **ebit**: utilidad antes de intereses e impuestos
- **depreciation**: depreciación
- **capex**: gastos de capital
- **ncwc**: cambio en capital de trabajo neto
- **bv**: (posiblemente) valor en libros
- **y**: variable objetivo que deseamos predecir (por ejemplo, un valor financiero o de valoración)

Iremos desde la **carga** y **exploración** del dataset hasta la **interpretación** de coeficientes y la **evaluación** de métricas importantes.

In [1]:
!wget https://github.com/javierherrera1996/IntroMachineLearning/raw/refs/heads/main/PrimerCorte/RegresionLineal/enterprise.csv

--2025-03-05 21:47:52--  https://github.com/javierherrera1996/IntroMachineLearning/raw/refs/heads/main/PrimerCorte/RegresionLineal/enterprise.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMachineLearning/refs/heads/main/PrimerCorte/RegresionLineal/enterprise.csv [following]
--2025-03-05 21:47:52--  https://raw.githubusercontent.com/javierherrera1996/IntroMachineLearning/refs/heads/main/PrimerCorte/RegresionLineal/enterprise.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82415 (80K) [text/plain]
Saving to: ‘enterprise.csv’

enterprise.csv      100%[==============

## Paso 0: Importar librerías y configuración

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 6)

## Paso 1: Carga y Exploración de Datos (EDA)

En este primer bloque, cargaremos nuestro dataset y realizaremos un Análisis Exploratorio básico.

In [11]:

df = pd.read_csv('enterprise.csv')
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061 entries, 0 to 1060
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   valuationdate  1061 non-null   object 
 1   revenues       1061 non-null   float64
 2   ebit           1061 non-null   float64
 3   depreciation   1061 non-null   float64
 4   capex          1061 non-null   float64
 5   ncwc           1061 non-null   float64
 6   bv             1061 non-null   float64
 7   y              1061 non-null   float64
dtypes: float64(7), object(1)
memory usage: 66.4+ KB


### Estadísticas Descriptivas y Distribuciones

In [12]:
df.describe()

,revenues,ebit,depreciation,capex,ncwc,bv,y
count,1.061000e+03,1061.000000,1061.000000,1061.000000,1061.000000,1.061000e+03,1.061000e+03
mean,1.632727e+05,24076.406810,8002.050367,10961.602205,22324.130946,8.057162e+04,3.292238e+05
std,2.213828e+05,31871.076559,13898.135849,20250.735197,54636.421540,1.043500e+05,4.056701e+05
min,8.560000e+01,-29982.979880,-5.690000,-694.661000,-118819.899000,-1.503965e+04,3.692000e+02
25%,4.265958e+04,4522.900000,999.340000,1666.600000,3115.247000,1.689500e+04,6.981010e+04
50%,1.082011e+05,12224.611950,3446.250000,4949.447000,10736.200000,4.172290e+04,1.850416e+05
75%,1.862485e+05,32228.941870,8361.220000,11201.479000,23454.185000,1.147563e+05,4.258273e+05
max,2.586712e+06,289541.300000,128993.500000,205525.700000,623588.700000,1.009026e+06,4.175892e+06


### Preguntas de Reflexión (Exploración)
1. ¿Qué tipo de problema estás intentando resolver al usar `y` como variable objetivo?
2. ¿Notas valores atípicos (outliers) en alguna de las variables? ¿Cómo podría afectar a la regresión lineal?
3. ¿Crees que la columna `valuationdate` podría aportar información para predecir `y`, o la excluirías del análisis?

1.-Valor de mercado, el objetivo es predecir ingresos.
2.-Rendimientos inusuales en ciertos periodos.
Cambios drásticos en el comportamiento de variables como los ingresos, gastos de capital.
3.-La columna valuationdate contiene fechas de valoración, lo cual podría tener algún valor temporal si hay patrones estacionales, tendencias o ciclos económicos que influencien la variable objetivo y. Sin embargo, depende de cómo esté estructurado el conjunto de datos

## Paso 2: Preparación y Limpieza de Datos

1. **Manejo de valores faltantes**.
2. **Posible eliminación** o **imputación** de outliers.
3. **Transformaciones** (escalado, log, etc.) si son necesarias.

In [13]:
for col in df.columns:
    if df[col].isnull().any():
        df[col].fillna(df[col].mean(), inplace=True)

for col in ['revenues', 'ebit', 'depreciation', 'capex', 'ncwc', 'bv', 'y']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
df.head()


,valuationdate,revenues,ebit,depreciation,capex,ncwc,bv,y
0,30/06/2011,86470.7000,10387.500000,6429.000,3512.000,16848.90000,47945.100,91760.80000
2,30/09/2016,15653.4800,221.347452,647.239,749.478,1468.42800,10128.726,23323.68520
3,30/09/2018,62001.3297,7551.761957,1808.830,2702.850,11873.25464,27144.930,130202.79240
4,30/09/2014,1242.8300,497.141501,232.027,1552.908,149.97100,1463.052,11472.12267
7,31/12/2012,21193.9000,3962.154660,824.900,1021.500,3864.00000,7917.500,11524.80000


### Pregunta de Reflexión (Limpieza)
¿Por qué es relevante tratar valores faltantes y/o outliers **antes** de entrenar el modelo de regresión lineal?

Es crucial tratar los valores faltantes y outliers antes de entrenar un modelo de regresión lineal porque estos problemas pueden afectar negativamente la precisión y robustez del modelo. Los valores faltantes pueden causar pérdida de información o sesgo si no se manejan adecuadamente, mientras que los outliers pueden distorsionar los coeficientes de la regresión, reduciendo la capacidad del modelo para generalizar correctamente. Al manejar estos problemas, se mejora el ajuste del modelo, se optimizan las predicciones y se evita el sobreajuste, lo que resulta en un modelo más confiable y con mejores resultados en datos no vistos.





## Paso 4: Selección de Variables e Implementación del Modelo

1. **Seleccionar** cuáles columnas usarás como variables explicativas (features). Puede que uses todas o hagas una selección.
2. **Asignar** `X` (features) y `y` (variable objetivo).
3. **Dividir** en conjunto de entrenamiento (train) y conjunto de prueba (test).

In [14]:
from sklearn.model_selection import train_test_split
X = df[['revenues', 'ebit', 'depreciation', 'capex', 'ncwc', 'bv']]
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Preguntas de Reflexión (Selección de Variables)
1. ¿Por qué es importante separar los datos en train y test?
2. ¿Qué pasaría si entrenas y evalúas en el **mismo** conjunto de datos?

Separar los datos en conjunto de entrenamiento (train) y conjunto de prueba (test) es crucial para evaluar de manera justa el rendimiento del modelo en datos no vistos. Si entrenas y evalúas en el mismo conjunto, el modelo podría **sobreajustarse (overfitting)**, lo que significa que aprendería detalles específicos del conjunto de entrenamiento y no generalizaría bien a nuevos datos, dando una falsa sensación de alta precisión.

## Paso 5: Entrenar el Modelo de Regresión Lineal
Usaremos la clase `LinearRegression` de scikit-learn.

In [18]:
# Crear instancia del modelo
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)


LinearRegression()

### Interpretación de Coeficientes
Si la lista de features es `["revenues", "ebit", "depreciation", "capex", "ncwc", "bv"]` y tu `model.coef_` es algo como `[a, b, c, d, e, f]`, entonces:

\[ y \approx \beta_0 + a\cdot revenues + b\cdot ebit + c\cdot depreciation + d\cdot capex + e\cdot ncwc + f\cdot bv \]

Cada coeficiente indica cómo cambia `y` en promedio **por cada unidad** que se incrementa la variable (manteniendo las demás constantes).

## Paso 6: Evaluación y Validación del Modelo
Usaremos las siguientes métricas:
- **MSE (Mean Squared Error)**
- **MAE (Mean Absolute Error)**
- **R² (Coeficiente de Determinación)**

In [19]:
# Predicciones en entrenamiento y prueba
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# MSE y R^2 en entrenamiento
mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

# MSE y R^2 en prueba
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("Conjunto de Entrenamiento:")
print(f"MSE: {mse_train:.2f}")
print(f"R^2: {r2_train:.2f}")
print("\nConjunto de Prueba:")
print(f"MSE: {mse_test:.2f}")
print(f"R^2: {r2_test:.2f}")


Conjunto de Entrenamiento:
MSE: 2242515099.11
R^2: 0.79

Conjunto de Prueba:
MSE: 2451454626.47
R^2: 0.75


### Preguntas de Reflexión (Evaluación)
1. ¿Por qué comparamos las métricas en **Entrenamiento** vs **Prueba**?
2. ¿Qué significa si el error en entrenamiento es **muy** distinto (mucho menor o mucho mayor) que en prueba?
3. ¿Cómo interpretar un **R²** cercano a 1, cercano a 0 o incluso negativo?

1.- ¿Por qué comparamos las métricas en Entrenamiento vs Prueba?
 Comparamos las métricas en entrenamiento vs prueba para evaluar la capacidad de generalización del modelo.  El rendimiento en el conjunto de entrenamiento nos indica qué tan bien el modelo se ajusta a los datos con los que fue entrenado.  Sin embargo, lo que realmente importa es cómo se comporta el modelo con datos nuevos que no ha visto antes.  El conjunto de prueba sirve como una simulación de datos reales, y comparando las métricas en ambos conjuntos podemos detectar si el modelo está sobreajustado (overfitting) o subajustado (underfitting).  Un modelo sobreajustado aprende demasiado bien los datos de entrenamiento, incluyendo el ruido, y por lo tanto, tiene un bajo error en entrenamiento pero un alto error en prueba.  Un modelo subajustado no captura la complejidad de los datos, por lo que tiene un error alto tanto en entrenamiento como en prueba.


2.- ¿Qué significa si el error en entrenamiento es muy distinto (mucho menor o mucho mayor) que en prueba?
Si el error en entrenamiento es mucho menor que en prueba, significa que el modelo está sobreajustado (overfitting). Ha memorizado los datos de entrenamiento, incluyendo el ruido, y no generaliza bien a nuevos datos.  En cambio, si el error en entrenamiento es similar al error en prueba, pero ambos son altos, indica que el modelo está subajustado (underfitting).  El modelo es demasiado simple para capturar la complejidad de los datos, por lo que no puede ajustarse bien ni a los datos de entrenamiento ni a los de prueba.


3.- ¿Cómo interpretar un R² cercano a 1, cercano a 0 o incluso negativo?
 El coeficiente de determinación (R²) mide la proporción de la varianza de la variable dependiente (y) que es explicada por el modelo.
-R² cercano a 1: Indica que el modelo ajusta muy bien los datos y explica una gran parte de la variabilidad de la variable objetivo.  En un escenario ideal, un R² de 1 significaría que el modelo predice perfectamente la variable objetivo.
-R² cercano a 0: Sugiere que el modelo no explica casi nada de la variabilidad de la variable objetivo, y es similar a usar simplemente la media de la variable objetivo como predicción.  Es una señal de un mal ajuste del modelo.
-R² negativo:  Implica que el modelo es peor que usar la media de la variable objetivo como predicción. Esto suele ocurrir cuando el modelo es inadecuado para los datos o hay problemas con los datos en sí, como la presencia de outliers.


### Pregunta de Reflexión (Residuos)
Si observas un patrón claro en la gráfica de residuos (por ejemplo, forma de U), ¿qué podría indicar respecto a las **asunciones** de la regresión lineal?

Un patrón en forma de U en la gráfica de residuos sugiere que la relación entre las variables independientes y la variable dependiente no es lineal, violando la asunción de linealidad de la regresión lineal.  La regresión lineal asume una relación lineal entre las variables predictoras y la variable respuesta. Una forma de U en los residuos indica que la relación es curvilínea, lo que significa que una regresión lineal no es el modelo adecuado. Un modelo no lineal (polinomial, por ejemplo) sería más apropiado para capturar la verdadera relación entre las variables.

### Preguntas de Reflexión Final
1. ¿Crees que la Regresión Lineal Simple/Múltiple **captura adecuadamente** las relaciones en tus datos?
2. ¿Qué otras técnicas (ej. árboles de decisión, bosques aleatorios, etc.) considerarías para comparar resultados?
3. ¿El dataset es suficientemente grande y variado para hacer conclusiones robustas?

1.- No necesariamente. La regresión lineal asume una relación lineal entre las variables predictoras y la variable objetivo.  Si la relación es no lineal, la regresión lineal no capturará adecuadamente la complejidad de los datos, y los residuos podrían mostrar un patrón sistemático (como una forma de U), como se mencionó antes. La exploración de los datos y el análisis de los residuos son cruciales para determinar la adecuación de la regresión lineal.  Un buen R^2 no siempre garantiza una buena representación de la realidad.

2.- **Árboles de decisión:** Son buenos para capturar relaciones no lineales y pueden manejar variables categóricas. Pueden ser más fáciles de interpretar que otros modelos. * **Bosques aleatorios:** Son un conjunto de árboles de decisión que trabajan en conjunto para reducir la varianza y el overfitting. Generalmente tienen un buen rendimiento y son robustos a outliers.* **Redes neuronales:** Son modelos más complejos capaces de aprender relaciones muy no lineales en los datos. Sin embargo, requieren más datos y ajustes de hiperparámetros.
 **Regresión polinomial:** Permite modelar relaciones curvilíneas entre las variables. **Support Vector Machines (SVM):** Son buenos para problemas de clasificación y regresión, y pueden ser muy efectivos en datos de alta dimensión.
La elección dependerá de la naturaleza de los datos y del problema específico.

3.-El tamaño del dataset proporcionado no es especificado, por lo que no se puede determinar si es lo suficientemente grande. Para conclusiones robustas, se necesita una muestra representativa y un tamaño suficiente para que el modelo generalice bien a datos no vistos. Un dataset pequeño puede llevar a un sobreajuste, y un dataset no representativo a resultados sesgados. La variabilidad de los datos también es clave: si la muestra tiene poca varianza, es posible que no se puedan identificar patrones y las conclusiones sean menos confiables.


## Resumen
En este Notebook hemos:
1. **Explorado** los datos (EDA) y detectado posibles outliers.
2. **Limpieza** de datos y transformaciones opcionales.
3. **Seleccionado** variables y dividido en train/test.
4. **Entrenado** un modelo de Regresión Lineal.
5. **Evaluado** con MSE, MAE, R² y analizado residuos.
6. **Reflexionado** sobre los coeficientes y conclusiones.

¡Has completado un **análisis de regresión lineal** de forma integral! Para afianzar, puedes:
- Ajustar parámetros (aunque la regresión lineal simple en sklearn no tiene muchos).
- Comparar con otros modelos.
- Realizar validación cruzada en lugar de una simple división train/test.

¡Éxitos en tu proyecto de análisis!